# Partie 2 : Transformation des Données (Version 4)

Ce notebook présente le processus de transformation des données brutes pour notre projet ETL sur les pandémies. Nous allons nettoyer, agréger, normaliser et supprimer les doublons des données extraites dans la phase précédente.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings

# Ignorer les avertissements
warnings.filterwarnings('ignore')

# Configuration pour afficher plus de colonnes
pd.set_option('display.max_columns', None)

## 1. Vérification des chemins et création du répertoire de sortie

In [ ]:
# Chemins des fichiers de données
covid_data_path = '../Partie-01-Extraction/Donnees Brutes/covid_19_clean_complete.csv'
monkeypox_data_path = '../Partie-01-Extraction/Donnees Brutes/owid-monkeypox-data.csv'
worldometer_data_path = '../Partie-01-Extraction/Donnees Brutes/worldometer_coronavirus_daily_data.csv'

# Vérification de l'existence des fichiers
files_exist = True
for file_path in [covid_data_path, monkeypox_data_path, worldometer_data_path]:
    if not os.path.exists(file_path):
        print(f"ERREUR: Le fichier {file_path} n'existe pas.")
        files_exist = False
    else:
        print(f"Le fichier {file_path} existe.")

if not files_exist:
    print("Certains fichiers de données sont manquants. Veuillez vérifier les chemins.")

# Création du répertoire de sortie
output_dir = '../Partie-03-Chargement/donnees_nettoyees/'
os.makedirs(output_dir, exist_ok=True)
print(f"Répertoire de sortie créé: {output_dir}")

## 2. Chargement des données brutes

In [ ]:
# Fonction pour charger un fichier CSV avec gestion des erreurs
def load_csv_file(file_path, file_desc):
    try:
        df = pd.read_csv(file_path)
        print(f"Données {file_desc} chargées avec succès. Forme: {df.shape}")
        display(df.head())
        return df
    except Exception as e:
        print(f"Erreur lors du chargement des données {file_desc}: {e}")
        return None

# Chargement des données COVID-19
covid_df = load_csv_file(covid_data_path, "COVID-19")

# Chargement des données Monkeypox
monkeypox_df = load_csv_file(monkeypox_data_path, "Monkeypox")

# Chargement des données Worldometer
worldometer_df = load_csv_file(worldometer_data_path, "Worldometer")

## 3. Nettoyage des données COVID-19

In [ ]:
def clean_covid_data(df):
    if df is None:
        return None
    
    # Copie du DataFrame
    cleaned_df = df.copy()
    
    # Conversion des dates
    cleaned_df['Date'] = pd.to_datetime(cleaned_df['Date'])
    
    # Normalisation des noms de colonnes
    cleaned_df.columns = [col.lower().replace(' ', '_') for col in cleaned_df.columns]
    
    # Remplacement des valeurs manquantes par 0 pour les colonnes numériques
    for col in cleaned_df.select_dtypes(include=['float64', 'int64']).columns:
        cleaned_df[col] = cleaned_df[col].fillna(0)
    
    # Suppression des doublons
    cleaned_df = cleaned_df.drop_duplicates()
    
    return cleaned_df

# Nettoyage des données COVID-19
covid_cleaned = clean_covid_data(covid_df)
if covid_cleaned is not None:
    print(f"Données COVID-19 nettoyées. Forme: {covid_cleaned.shape}")
    display(covid_cleaned.head())

## 4. Nettoyage des données Monkeypox

In [ ]:
def clean_monkeypox_data(df):
    if df is None:
        return None
    
    # Copie du DataFrame
    cleaned_df = df.copy()
    
    # Conversion des dates
    if 'date' in cleaned_df.columns:
        cleaned_df['date'] = pd.to_datetime(cleaned_df['date'])
    
    # Normalisation des noms de colonnes
    cleaned_df.columns = [col.lower().replace(' ', '_') for col in cleaned_df.columns]
    
    # Remplacement des valeurs manquantes par 0 pour les colonnes numériques
    for col in cleaned_df.select_dtypes(include=['float64', 'int64']).columns:
        cleaned_df[col] = cleaned_df[col].fillna(0)
    
    # Suppression des doublons
    cleaned_df = cleaned_df.drop_duplicates()
    
    return cleaned_df

# Nettoyage des données Monkeypox
monkeypox_cleaned = clean_monkeypox_data(monkeypox_df)
if monkeypox_cleaned is not None:
    print(f"Données Monkeypox nettoyées. Forme: {monkeypox_cleaned.shape}")
    display(monkeypox_cleaned.head())

## 5. Nettoyage des données Worldometer

In [ ]:
def clean_worldometer_data(df):
    if df is None:
        return None
    
    # Copie du DataFrame
    cleaned_df = df.copy()
    
    # Conversion des dates
    if 'date' in cleaned_df.columns:
        cleaned_df['date'] = pd.to_datetime(cleaned_df['date'])
    
    # Normalisation des noms de colonnes
    cleaned_df.columns = [col.lower().replace(' ', '_') for col in cleaned_df.columns]
    
    # Remplacement des valeurs manquantes par 0 pour les colonnes numériques
    for col in cleaned_df.select_dtypes(include=['float64', 'int64']).columns:
        cleaned_df[col] = cleaned_df[col].fillna(0)
    
    # Suppression des doublons
    cleaned_df = cleaned_df.drop_duplicates()
    
    return cleaned_df

# Nettoyage des données Worldometer
worldometer_cleaned = clean_worldometer_data(worldometer_df)
if worldometer_cleaned is not None:
    print(f"Données Worldometer nettoyées. Forme: {worldometer_cleaned.shape}")
    display(worldometer_cleaned.head())

## 6. Préparation des tables pour le chargement

In [ ]:
# Préparation de la table des localisations
def prepare_locations():
    # Création d'une liste pour stocker les localisations
    locations = []
    
    # Extraction des localisations depuis COVID-19
    if covid_cleaned is not None and 'country_region' in covid_cleaned.columns:
        covid_locs = covid_cleaned[['country_region']].drop_duplicates()
        covid_locs = covid_locs.rename(columns={'country_region': 'pays'})
        locations.append(covid_locs)
    
    # Extraction des localisations depuis Monkeypox
    if monkeypox_cleaned is not None and 'entity' in monkeypox_cleaned.columns:
        mpox_locs = monkeypox_cleaned[['entity']].drop_duplicates()
        mpox_locs = mpox_locs.rename(columns={'entity': 'pays'})
        locations.append(mpox_locs)
    
    # Fusion et suppression des doublons
    if locations:
        locations_df = pd.concat(locations, ignore_index=True)
        locations_df = locations_df.drop_duplicates()
        
        # Ajout des colonnes nécessaires selon le schéma SQL
        locations_df['id_localisation'] = range(1, len(locations_df) + 1)
        locations_df['code_pays'] = None
        locations_df['region'] = None
        locations_df['continent'] = None
        locations_df['latitude'] = None
        locations_df['longitude'] = None
        locations_df['population'] = None
        
        # Réorganisation des colonnes selon le schéma SQL
        locations_df = locations_df[['id_localisation', 'pays', 'code_pays', 'region', 'continent', 'latitude', 'longitude', 'population']]
        
        return locations_df
    else:
        return pd.DataFrame(columns=['id_localisation', 'pays', 'code_pays', 'region', 'continent', 'latitude', 'longitude', 'population'])

# Préparation de la table des pandémies
def prepare_pandemics():
    # Création manuelle de la table des pandémies selon le schéma SQL
    pandemics_df = pd.DataFrame({
        'id_pandemie': [1, 2],
        'nom': ['COVID-19', 'Monkeypox'],
        'agent_pathogene': ['SARS-CoV-2', 'Monkeypox virus'],
        'description': [
            'Maladie respiratoire causée par le coronavirus SARS-CoV-2 identifiée pour la première fois à Wuhan Chine en décembre 2019',
            'Maladie virale zoonotique causée par le virus de la variole du singe appartenant au genre Orthopoxvirus'
        ],
        'date_debut': ['2019-12-01', '2022-05-01'],
        'date_fin': [None, None]
    })
    
    return pandemics_df

# Création des tables
locations_df = prepare_locations()
pandemics_df = prepare_pandemics()

print(f"Table des localisations créée avec {len(locations_df)} entrées.")
display(locations_df.head())

print(f"Table des pandémies créée avec {len(pandemics_df)} entrées.")
display(pandemics_df)

## 7. Préparation des données pour le chargement

In [ ]:
# Préparation des données COVID-19 pour le chargement
def prepare_covid_data_for_loading():
    if covid_cleaned is None:
        return None
    
    # Sélection des colonnes pertinentes
    if all(col in covid_cleaned.columns for col in ['date', 'country_region', 'confirmed', 'deaths', 'recovered', 'active']):
        covid_data = covid_cleaned[['date', 'country_region', 'confirmed', 'deaths', 'recovered', 'active']].copy()
        
        # Renommage des colonnes
        covid_data = covid_data.rename(columns={
            'date': 'date',
            'country_region': 'pays',
            'confirmed': 'cas_confirmes',
            'deaths': 'deces',
            'recovered': 'guerisons',
            'active': 'cas_actifs'
        })
        
        # Ajout de la colonne pandémie
        covid_data['id_pandemie'] = 1
        
        return covid_data
    else:
        print("Colonnes requises manquantes dans les données COVID-19.")
        return None

# Préparation des données Monkeypox pour le chargement
def prepare_monkeypox_data_for_loading():
    if monkeypox_cleaned is None:
        return None
    
    # Sélection des colonnes pertinentes
    if all(col in monkeypox_cleaned.columns for col in ['date', 'entity', 'confirmed_cases']):
        mpox_data = monkeypox_cleaned[['date', 'entity', 'confirmed_cases']].copy()
        
        # Renommage des colonnes
        mpox_data = mpox_data.rename(columns={
            'date': 'date',
            'entity': 'pays',
            'confirmed_cases': 'cas_confirmes'
        })
        
        # Ajout des colonnes manquantes
        mpox_data['deces'] = 0
        mpox_data['guerisons'] = 0
        mpox_data['cas_actifs'] = mpox_data['cas_confirmes']
        
        # Ajout de la colonne pandémie
        mpox_data['id_pandemie'] = 2
        
        return mpox_data
    else:
        print("Colonnes requises manquantes dans les données Monkeypox.")
        return None

# Préparation des données finales
covid_final = prepare_covid_data_for_loading()
mpox_final = prepare_monkeypox_data_for_loading()

# Affichage des données préparées
if covid_final is not None:
    print(f"Données COVID-19 préparées: {covid_final.shape}")
    display(covid_final.head())
    
if mpox_final is not None:
    print(f"Données Monkeypox préparées: {mpox_final.shape}")
    display(mpox_final.head())

## 8. Enregistrement des données transformées

In [ ]:
# Fonction pour enregistrer les données transformées
def save_data(df, filename):
    if df is not None:
        filepath = os.path.join(output_dir, filename)
        df.to_csv(filepath, index=False)
        print(f"Données enregistrées dans {filepath}")
        return True
    else:
        print(f"Impossible d'enregistrer {filename} car les données sont None.")
        return False

# Enregistrement des tables de dimension
save_data(locations_df, 'localisations_clean.csv')
save_data(pandemics_df, 'pandemies_clean.csv')

# Enregistrement des données spécifiques pour la phase de chargement
if covid_final is not None:
    # Cas confirmés COVID-19
    cas_confirmes_covid = covid_final[['date', 'pays', 'cas_confirmes', 'id_pandemie']].copy()
    save_data(cas_confirmes_covid, 'cas_confirmes_clean.csv')
    
    # Décès COVID-19
    deces_covid = covid_final[['date', 'pays', 'deces', 'id_pandemie']].copy()
    save_data(deces_covid, 'deces_clean.csv')
    
    # Guérisons COVID-19
    guerisons_covid = covid_final[['date', 'pays', 'guerisons', 'id_pandemie']].copy()
    save_data(guerisons_covid, 'guerisons_clean.csv')

# Ajout des données Monkeypox aux fichiers existants
if mpox_final is not None:
    # Cas confirmés Monkeypox
    cas_confirmes_mpox = mpox_final[['date', 'pays', 'cas_confirmes', 'id_pandemie']].copy()
    cas_confirmes_file = os.path.join(output_dir, 'cas_confirmes_clean.csv')
    
    if os.path.exists(cas_confirmes_file):
        # Lire le fichier existant
        cas_confirmes_df = pd.read_csv(cas_confirmes_file)
        # Concaténer avec les nouvelles données
        cas_confirmes_df = pd.concat([cas_confirmes_df, cas_confirmes_mpox], ignore_index=True)
        # Enregistrer le fichier mis à jour
        cas_confirmes_df.to_csv(cas_confirmes_file, index=False)
        print(f"Données Monkeypox ajoutées à {cas_confirmes_file}")
    else:
        save_data(cas_confirmes_mpox, 'cas_confirmes_clean.csv')
    
    # Décès Monkeypox
    deces_mpox = mpox_final[['date', 'pays', 'deces', 'id_pandemie']].copy()
    deces_file = os.path.join(output_dir, 'deces_clean.csv')
    
    if os.path.exists(deces_file):
        deces_df = pd.read_csv(deces_file)
        deces_df = pd.concat([deces_df, deces_mpox], ignore_index=True)
        deces_df.to_csv(deces_file, index=False)
        print(f"Données Monkeypox ajoutées à {deces_file}")
    else:
        save_data(deces_mpox, 'deces_clean.csv')
    
    # Guérisons Monkeypox
    guerisons_mpox = mpox_final[['date', 'pays', 'guerisons', 'id_pandemie']].copy()
    guerisons_file = os.path.join(output_dir, 'guerisons_clean.csv')
    
    if os.path.exists(guerisons_file):
        guerisons_df = pd.read_csv(guerisons_file)
        guerisons_df = pd.concat([guerisons_df, guerisons_mpox], ignore_index=True)
        guerisons_df.to_csv(guerisons_file, index=False)
        print(f"Données Monkeypox ajoutées à {guerisons_file}")
    else:
        save_data(guerisons_mpox, 'guerisons_clean.csv')

print("\nToutes les données ont été enregistrées dans le répertoire de sortie.")

## 9. Vérification des fichiers générés

In [ ]:
# Vérification des fichiers générés
output_files = os.listdir(output_dir)
print(f"Fichiers générés dans {output_dir}:")
for file in output_files:
    file_path = os.path.join(output_dir, file)
    file_size = os.path.getsize(file_path) / 1024  # Taille en Ko
    print(f"- {file} ({file_size:.2f} Ko)")
    
    # Afficher les premières lignes de chaque fichier
    df = pd.read_csv(file_path)
    print(f"  Nombre de lignes: {len(df)}")
    display(df.head(3))
    print("\n")

## 10. Prochaine étape

Maintenant que les données ont été transformées et enregistrées dans les fichiers CSV appropriés, vous pouvez passer à la phase de chargement en exécutant le notebook `Chargement_v2.ipynb` dans le dossier `Partie-03-Chargement`.

Assurez-vous que votre base de données MySQL est configurée correctement avant d'exécuter ce notebook. Les paramètres de connexion sont :
- Nom de la base de données: epiviz
- Utilisateur: root
- Mot de passe: (aucun)
- Hôte: localhost
- Port: 3306